 <span style="color: #000508; font-family: Segoe UI; font-size: 2.0em; font-weight: 300;">Online installations</span>

In [ ]:
# !pip install pycocotools>=2.0.2
# !pip install timm>=0.3.2
# !pip install omegaconf>=2.0
# !pip install ensemble-boxes

 <span style="color: #000508; font-family: Segoe UI; font-size: 2.0em; font-weight: 300;">Offline installations</span>

In [3]:
## Install omegaconf - dependancy for effdet
!pip install --no-deps ../input/effdet-latestvinbigdata-wbf-fused/omegaconf-2.0.6-py3-none-any.whl

## Install timm & ensemble_boxes
!pip install --no-deps ../input/effdet-latestvinbigdata-wbf-fused/timm-0.3.4-py3-none-any.whl
!pip install --no-deps ../input/effdet-latestvinbigdata-wbf-fused/ensemble_boxefs-1.0.4-py3-none-any.whl

!cp -r ../input/effdet-latestvinbigdata-wbf-fused/pycocotools-2.0.2/ .
!cd ./pycocotools-2.0.2 && python setup.py install
!rm -r pycocotools-2.0.2

!cp -r ../input/effdet-latestvinbigdata-wbf-fused/efficientdet-pytorch/ .
!cd ./efficientdet-pytorch && python setup.py install
!rm -r efficientdet-pytorch

In [ ]:
# Release GPU Memory.
from numba import cuda as CU
try:
    device = CU.get_current_device()
    device.reset()
except Exception as E:
    print("GPU not enabled. Nothing to clear and good to go.")


## Restart session to detect installed libraries
!pip list | grep effdet
import os
os._exit(00)
## Check PyTorch Version

<span style="color: #000508; font-family: Segoe UI; font-size: 2.0em; font-weight: 300;">Import Packages</span>

In [1]:
import sys
import torch
import os
from datetime import datetime
import time
import random
import cv2
import gc
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from glob import glob
import warnings
from collections import Counter

#from ensemble_boxes import weighted_boxes_fusion
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.utils.data.dataloader import default_collate

from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain,DetBenchPredict
from effdet.efficientdet import HeadNet
from effdet import create_model, unwrap_bench, create_loader, create_dataset, create_evaluator, create_model_from_config
from effdet.data import resolve_input_config, SkipSubset
from effdet.anchors import Anchors, AnchorLabeler
from timm.models import resume_checkpoint, load_checkpoint
from timm.utils import *
from timm.optim import create_optimizer
from timm.scheduler import create_scheduler

<span style="color: #000508; font-family: Segoe UI; font-size: 2.0em; font-weight: 300;">Seed Everything</span>


In [2]:
SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(SEED)

In [3]:
train = os.listdir('../input/originaldataset/original_dataset_split/original_dataset_split/train/school')
val = os.listdir('../input/originaldataset/original_dataset_split/original_dataset_split/val/school')
test = os.listdir('../input/originaldataset/original_dataset_split/original_dataset_split/test/school')

<span style="color: #000508; font-family: Segoe UI; font-size: 2.0em; font-weight: 300;">Load Data</span>

In [4]:
data = pd.read_csv('../input/schooldata/school_data/final_data.csv')
data.rename(columns={'filename':'image_id', 'x1':'x_min', 'y1':'y_min', 'x2':'x_max', 'y2':'y_max'}, inplace=True)
data.head()

In [5]:
data['split'] = 'na'
data.loc[data[data['image_id'].isin(train)].index, 'split'] = 'train' 
data.loc[data[data['image_id'].isin(val)].index, 'split'] = 'valid'
data.loc[data[data['image_id'].isin(test)].index, 'split']  = 'test'

In [9]:
data['split'].value_counts()

In [6]:
#data['image_path'] = data['image_id'].map(lambda x:os.path.join('../input/school-data2/school_data/schools_annotated',str(x)))
data['class_id'] = [1] * data.shape[0]
data.head()

In [7]:
data = data[~data['image_id'].isin(['42201308.png','61212325.png', '61223303.png'])]
data.drop_duplicates(subset=['image_id'], keep='first', inplace=True)
data.reset_index(drop=True, inplace=True)
data.drop(columns='id', inplace=True)

In [8]:
train_ns = []
secd = data[data['split']=='train']['image_id'].tolist()
for elem in train:
    if elem not in secd:
        train_ns.append(elem)
val_ns = []
secd = data[data['split']=='valid']['image_id'].tolist()
for elem in val:
    if elem not in secd:
        val_ns.append(elem)
test_ns = []
secd = data[data['split']=='test']['image_id'].tolist()
for elem in test:
    if elem not in secd:
        test_ns.append(elem)

In [9]:
split_train = ['train'] * len(train_ns)
split_val = ['valid'] * len(val_ns)
split_test = ['test'] * len(test_ns)
final_split = split_train + split_val + split_test
final_ns = train_ns + val_ns + test_ns
x_min = [0] * len(final_ns)
y_min = [0] * len(final_ns)
x_max = [256] * len(final_ns)
y_max = [256] * len(final_ns)
class_id = [0] * len(final_ns)

In [10]:
data2 = pd.DataFrame()
data2['x_min'] = x_min
data2['y_min'] = y_min
data2['x_max'] = x_max
data2['y_max'] = y_max
data2['image_id'] = final_ns
data2['split'] = final_split
data2['class_id'] = class_id
data2.head()

In [11]:
data2 = data2[~data2['image_id'].isin(['42304318_E.png', '41606305_N.png','11302316_E.png', '11301320_E.png', 'train.json', 'test.json', 'val.json', '23301340_W.png', '42201308.png'])]


In [12]:
data = pd.concat([data, data2], ignore_index=True)

In [13]:
data.shape
data['split'].value_counts()

In [14]:
data = data[~data['image_id'].isin(['61212325.png', '61223303.png'])]

<span style="color: #0087e4; font-family: Segoe UI; font-size: 2.3em; font-weight: 300;">Augmentations & Transforms</span>

Usign the Albumentations package to create an augmentation pipeline for training and validation. 

In [15]:
def get_train_transforms():
    return A.Compose(
        [
        A.Resize(height=512, width=512, p=1),
        ToTensorV2(p=1.0)
        ], 
        p=1.0, 
        bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels']
        )
    )

def get_valid_transforms():
    return A.Compose(
        [
            A.Resize(height=512, width=512, p=1.0),
            ToTensorV2(p=1.0),
        ], 
        p=1.0, 
        bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels']
        )
    )

def get_test_transform():
    return A.Compose([
            A.Resize(height=512, width=512, p=1.0),
            ToTensorV2(p=1.0)], 
            p=1.0, 
            bbox_params=A.BboxParams(
                format='pascal_voc',
                min_area=0, 
                min_visibility=0,
                label_fields=['labels']
            ))

<span style="color: #0087e4; font-family: Segoe UI; font-size: 2.3em; font-weight: 300;">Dataset Retrieval and Pre-processsing</span>

Define the data loader class to retrive images, perform albumentation-based + custom CutMix and MixUp augmentations

In [16]:
class DatasetRetriever(Dataset):

    def __init__(self, marking, image_ids, TRAIN_ROOT_PATH, transforms=None, test=False):
        super().__init__()
        self.image_ids = image_ids
        self.marking = marking
        self.transforms = transforms
        self.test = test
        self.TRAIN_ROOT_PATH = TRAIN_ROOT_PATH
        
        
    def load_image_and_boxes(self, index):
        image_id = self.image_ids[index]
        #print(image_id)
        image = cv2.imread(f'{self.TRAIN_ROOT_PATH}/{image_id}', cv2.IMREAD_COLOR).copy()
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        records = self.marking[self.marking['image_id'] == image_id]
        boxes = records[['x_min', 'y_min', 'x_max', 'y_max']].values
        labels = records['class_id'].tolist()
        resize_transform = A.Compose([A.Resize(height=512, width=512, p=1.0)], 
                                    p=1.0, 
                                    bbox_params=A.BboxParams(
                                        format='pascal_voc',
                                        min_area=0.1, 
                                        min_visibility=0.1,
                                        label_fields=['labels'])
                                    )

        resized = resize_transform(**{
                'image': image,
                'bboxes': boxes,
                'labels': labels
            })

        resized_bboxes = np.vstack((list(bx) for bx in resized['bboxes']))
        return resized['image'], resized_bboxes, resized['labels']

    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
    
        image, boxes, labels = self.load_image_and_boxes(index)
        ## To prevent ValueError: y_max is less than or equal to y_min for bbox from albumentations bbox_utils
        labels = np.array(labels, dtype=np.int).reshape(len(labels), 1)
        combined = np.hstack((boxes.astype(np.int), labels))
        combined = combined[np.logical_and(combined[:,2] > combined[:,0],
                                                          combined[:,3] > combined[:,1])]
        boxes = combined[:, :4]
        labels = combined[:, 4].tolist()
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = torch.tensor(labels)
        target['image_id'] = torch.tensor([index])
        if self.transforms:
            for i in range(10):
                sample = self.transforms(**{
                    'image': image,
                    'bboxes': target['boxes'],
                    'labels': labels
                })
                if len(sample['bboxes']) > 0:
                    image = sample['image']
                    target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)
                    target['boxes'][:,[0,1,2,3]] = target['boxes'][:,[1,0,3,2]]  ## ymin, xmin, ymax, xmax
                    break
            
            ## Handling case where no valid bboxes are present
            if len(target['boxes'])==0 or i==9:
                return None
            else:
                ## Handling case where augmentation and tensor conversion yields no valid annotations
                try:
                    assert torch.is_tensor(image), f"Invalid image type:{type(image)}"
                    assert torch.is_tensor(target['boxes']), f"Invalid target type:{type(target['boxes'])}"
                except Exception as E:
                    print("Image skipped:", E)
                    return None      

        return image, target, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]
    
    

In [ ]:
# sample = data[data['split']=='valid']

# for i in range(sample.shape[0]):
#     #try:
#     DatasetRetriever(sample, sample['image_id'].tolist(), '../input/originaldataset/original_dataset_split/original_dataset_split/val/school')[i]
    

<span style="color: #000508; font-family: Segoe UI; font-size: 2.0em; font-weight: 300;">Helper Functions</span>

Function to store Average and Current values

In [17]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

<span style="color: #0087e4; font-family: Segoe UI; font-size: 2.3em; font-weight: 300;">Build Training and Validation Loops</span>

In [18]:
class Fitter:
    def __init__(self, model, device, config):
        self.config = config
        self.epoch = 0

        self.base_dir = f'./{config.folder}'
        
        if not os.path.exists(self.base_dir):
            os.makedirs(self.base_dir)
        
        self.log_path = f'{self.base_dir}/log.txt'
        self.best_summary_loss = 10**5
        self.model = model
        self.device = device

        param_optimizer = list(self.model.named_parameters())
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ] 

        self.optimizer = config.OptimizerClass(self.model.parameters(), lr=config.lr)
        self.scheduler = config.SchedulerClass(self.optimizer, **config.scheduler_params)
        self.log(f'Fitter prepared. Device is {self.device}')

    def fit(self, train_loader, validation_loader):
        history_dict = {}
        history_dict['epoch'] = []
        history_dict['train_loss'] = []
        history_dict['val_loss'] = []
        history_dict['train_lr'] = []
        
        for e in range(self.config.n_epochs):
            history_dict['epoch'].append(self.epoch)
            lr = self.optimizer.param_groups[0]['lr']
            timestamp = datetime.utcnow().isoformat()
            
            if self.config.verbose:
                self.log(f'\n{timestamp}\nLR: {lr}')

            t = time.time()
            summary_loss, loss_trend, lr_trend = self.train_epoch(train_loader)
            history_dict['train_loss'].append(loss_trend)
            history_dict['train_lr'].append(lr_trend)
            self.log(f'[RESULT]: Train. Epoch: {self.epoch}, summary_loss: {summary_loss.avg:.5f}, time: {(time.time() - t):.5f}')
            self.save(f'{self.base_dir}/last-checkpoint.bin')
            
            t = time.time()
            summary_loss, loss_trend = self.validation(validation_loader)
            history_dict['val_loss'].append(loss_trend)
            self.log(f'[RESULT]: Val. Epoch: {self.epoch}, summary_loss: {summary_loss.avg:.5f}, time: {(time.time() - t):.5f}')
            
            if summary_loss.avg < self.best_summary_loss:
                self.best_summary_loss = summary_loss.avg
                self.model.eval()
                self.save(f'{self.base_dir}/best-checkpoint-{str(self.epoch).zfill(3)}epoch.bin')
                
                try:
                    os.remove(f)
                except:pass
                f = f'{self.base_dir}/best-checkpoint-{str(self.epoch).zfill(3)}epoch.bin'

            if self.config.validation_scheduler:
                self.scheduler.step(metrics=summary_loss.avg)

            self.epoch += 1
        return history_dict

    def train_epoch(self, train_loader):
        self.model.train()
        summary_loss = AverageMeter()
        t = time.time()
        loss_trend = []
        lr_trend = []
        for step, (images, targets, image_ids) in tqdm(enumerate(train_loader), total=len(train_loader)):
            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    print(
                        f'Train Step {step}/{len(train_loader)}, ' + \
                        f'summary_loss: {summary_loss.avg:.5f}, ' + \
                        f'time: {(time.time() - t):.5f}', end='\r'
                    )            
            
            images = torch.stack(images)
            images = images.to(self.device).float()
            target_res = {}
            boxes = [target['boxes'].to(self.device).float() for target in targets]
            labels = [target['labels'].to(self.device).float() for target in targets]
            target_res['bbox'] = boxes
            target_res['cls'] = labels
            self.optimizer.zero_grad()
            output = self.model(images, target_res)

            loss = output['loss']
            loss.backward()
            summary_loss.update(loss.detach().item(), self.config.batch_size)
            self.optimizer.step()

            if self.config.step_scheduler:
                self.scheduler.step()

            
            lr = self.optimizer.param_groups[0]['lr']
            loss_trend.append(summary_loss.avg)
            lr_trend.append(lr)
        return summary_loss, loss_trend, lr_trend
    
    def validation(self, val_loader):
        self.model.eval()
        summary_loss = AverageMeter()
        t = time.time()
        loss_trend = []
#         lr_trend = []
        
        for step, (images, targets, image_ids) in tqdm(enumerate(val_loader), total=len(val_loader)):
            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    print(
                        f'Val Step {step}/{len(val_loader)}, ' + \
                        f'summary_loss: {summary_loss.avg:.5f}, ' + \
                        f'time: {(time.time() - t):.5f}', end='\r'
                    )
            with torch.no_grad():
                images = torch.stack(images)
                images = images.to(self.device).float()
                target_res = {}
                boxes = [target['boxes'].to(self.device).float() for target in targets]
                labels = [target['labels'].to(self.device).float() for target in targets]
                target_res['bbox'] = boxes
                target_res['cls'] = labels 
                target_res["img_scale"] = torch.tensor([1.0] * self.config.batch_size,
                                                       dtype=torch.float).to(self.device)
                target_res["img_size"] = torch.tensor([images[0].shape[-2:]] * self.config.batch_size,
                                                      dtype=torch.float).to(self.device)
                
                output = self.model(images, target_res)
        
                loss = output['loss']
                summary_loss.update(loss.detach().item(), self.config.batch_size)

                loss_trend.append(summary_loss.avg)
        return summary_loss, loss_trend[-1]
    
    def save(self, path):
        self.model.eval()
        torch.save({
            'model_state_dict': self.model.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict(),
            'best_summary_loss': self.best_summary_loss,
            'epoch': self.epoch,
        }, path)

    def load(self, path):
        checkpoint = torch.load(path)
        self.model.model.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        self.best_summary_loss = checkpoint['best_summary_loss']
        self.epoch = checkpoint['epoch'] + 1
        
    def log(self, message):
        if self.config.verbose:
            print(message)
        with open(self.log_path, 'a+') as logger:
            logger.write(f'{message}\n')

<span style="color: #0087e4; font-family: Segoe UI; font-size: 2.3em; font-weight: 300;">Define Training Configuration</span>

In [19]:
class TrainGlobalConfig:
    def __init__(self):
        self.num_classes = 2
        self.num_workers = 4
        self.batch_size = 4 
        self.n_epochs = 10
        self.lr = 0.0002
        self.model_name = 'tf_efficientdet_d1'
        self.folder = 'training_job'
        self.verbose = True
        self.verbose_step = 1
        self.step_scheduler = True
        self.validation_scheduler = False
        self.n_img_count = len(data.image_id.unique())
        self.OptimizerClass = torch.optim.AdamW
        self.SchedulerClass = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts
        self.scheduler_params = dict(
                            T_0=50,
                            T_mult=1,
                            eta_min=0.0001,
                            last_epoch=-1,
                            verbose=False
                            )
        self.kfold = 2
    
    def reset(self):
        self.OptimizerClass = torch.optim.AdamW
        self.SchedulerClass = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts

train_config = TrainGlobalConfig()

<span style="color: #0087e4; font-family: Segoe UI; font-size: 2.3em; font-weight: 300;">Initiate Training</span>

In [20]:
def collate_fn(batch):
    batch = list(filter(lambda x: x is not None, batch))
    return tuple(zip(*batch))

In [25]:
checkpoint_path = None
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")

train_ids = data[data['split'] == 'train'].image_id.unique()
val_ids = data[data['split'] == 'valid'].image_id.unique()


# train_ids = train_ids[:10]
# val_ids = val_ids[:10]
train_dataset = DatasetRetriever(
                    image_ids=train_ids,
                    marking=data,
                    TRAIN_ROOT_PATH='../input/originaldataset/original_dataset_split/original_dataset_split/train/school',
                    transforms=get_train_transforms(),
                    test=False,
                    )

validation_dataset = DatasetRetriever(
                        image_ids=val_ids,
                        marking=data,
                        TRAIN_ROOT_PATH='../input/originaldataset/original_dataset_split/original_dataset_split/val/school',
                        transforms=get_valid_transforms(),
                        test=True,
                        )

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=train_config.batch_size,
    sampler=RandomSampler(train_dataset),
    pin_memory=False,
    drop_last=True,
    num_workers=train_config.num_workers,
    collate_fn=collate_fn,
)
val_loader = torch.utils.data.DataLoader(
    validation_dataset, 
    batch_size=train_config.batch_size,
    num_workers=train_config.num_workers,
    shuffle=False,
    sampler=SequentialSampler(validation_dataset),
    pin_memory=False,
    collate_fn=collate_fn,
)
    

if(checkpoint_path):
    print(f'Resuming from checkpoint: {checkpoint_path}')        
    model = create_model_from_config(base_config, bench_task='train', bench_labeler=True,
                             num_classes=train_config.num_classes,
                             pretrained=False)
    model.to(device)

    fitter = Fitter(model=model, device=device, config=train_config)
    fitter.load(checkpoint_path)

else:
    
    config = get_efficientdet_config('tf_efficientdet_d1')
    config.num_classes = 1
    config.image_size = (512, 512)
    net = EfficientDet(config, pretrained_backbone=True)
    net.class_net = HeadNet(
        config,
        num_outputs=config.num_classes,
    )
    model = DetBenchTrain(net, config)
    model.to(device)

    fitter = Fitter(model=model, device=device, config=train_config)  
        
    model_config = model.config
    history_dict = fitter.fit(train_loader, val_loader)

In [ ]:
#Inference

In [21]:
data[(data['split'] == 'test')&(data['class_id']==0)].head()

In [35]:
len(test_ids)

In [34]:
#test_ids = data[(data['split'] == 'test')&(data['class_id']==0)].image_id.unique()
test_ids = data[(data['split'] == 'test')].image_id.unique()
test_dataset = DatasetRetriever(
                            image_ids=test_ids,
                            marking=data,
                            TRAIN_ROOT_PATH='../input/originaldataset/original_dataset_split/original_dataset_split/test/school',
                            transforms=get_test_transform(),
                            test=True,
                            )
test_loader = DataLoader(test_dataset,
                         batch_size = 4,
                         shuffle = False,
                         drop_last = False,
                         collate_fn = collate_fn) 

In [55]:
# valid_ids = data[data['split'] == 'valid'].image_id.unique()
# test_dataset = DatasetRetriever(
#                             image_ids=valid_ids,
#                             marking=data,
#                             TRAIN_ROOT_PATH='../input/originaldataset/original_dataset_split/original_dataset_split/val/school',
#                             transforms=get_valid_transforms(),
#                             test=True,
#                             )
# test_loader = DataLoader(test_dataset,
#                          batch_size = 4,
#                          shuffle = False,
#                          drop_last = False,
#                          collate_fn = collate_fn) 

In [25]:
!pip install git+https://github.com/alexhock/object-detection-metrics

In [26]:
!pip install ensemble-boxes

In [27]:
from ensemble_boxes import ensemble_boxes_wbf
def run_wbf(predictions, image_size=512, iou_thr=0.44, skip_box_thr=0.43, weights=None):
    bboxes = []
    confidences = []
    class_labels = []

    for prediction in predictions:
        boxes = [(prediction["boxes"] / image_size).tolist()]
        scores = [prediction["scores"].tolist()]
        labels = [prediction["classes"].tolist()]

        boxes, scores, labels = ensemble_boxes_wbf.weighted_boxes_fusion(
            boxes,
            scores,
            labels,
            weights=weights,
            iou_thr=iou_thr,
            skip_box_thr=skip_box_thr,
        )
        boxes = boxes * (image_size - 1)
        bboxes.append(boxes.tolist())
        confidences.append(scores.tolist())
        class_labels.append(labels.tolist())
    return bboxes, confidences, class_labels


In [56]:
from objdetecteval.metrics.coco_metrics import get_coco_stats
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
config = get_efficientdet_config('tf_efficientdet_d1')
config.num_classes = 1
config.image_size = (512, 512)
net = EfficientDet(config, pretrained_backbone=True)
net.class_net = HeadNet(
    config,
    num_outputs=config.num_classes,
)

checkpoint = torch.load('../input/checkpoints/best-checkpoint-004epoch.bin')
net.load_state_dict(checkpoint['model_state_dict'])

del checkpoint
gc.collect()
net = DetBenchPredict(net)
net.eval()
net = net.cuda()
result = []
truth_image_ids = []
truth_boxes = []
truth_labels = []
predicted_labels = []
predicted_boxes = []
predicted_confidences = []
image_ids = []
for step, (images, targets, image_ids) in tqdm(enumerate(test_loader), total=len(test_loader)):
    with torch.no_grad():
        images = torch.stack(images)
        images = images.to(device).float()
        target_res = {}

        boxes = [target['boxes'][:, [1, 0, 3, 2]].detach().tolist() for target in targets]
        labels = [target['labels'].detach().tolist() for target in targets]
        
        target_res["img_scale"] = torch.tensor([1.0] * 4,
                                               dtype=torch.float).to(device)
        target_res["img_size"] = torch.tensor([images[0].shape[-2:]] * 4,
                                              dtype=torch.float).to(device)
        truth_boxes.extend(boxes)
        truth_labels.extend(labels)
        
        output = net(images, {'img_scale':target_res["img_scale"], 'img_size':target_res["img_size"]})
        for i in range(images.shape[0]):
            boxes = output[i].detach().cpu().numpy()[:,:4]
            scores = output[i].detach().cpu().numpy()[:,4]
            classes = output[i].detach().cpu().numpy()[:, 5]
            
            indexes = np.where(scores > 0.4)[0]
            result.append({
                'boxes': boxes[indexes],
                'scores': scores[indexes],
                'classes': classes[indexes],
            })
            #print(scores)
            boxes, scores, labels = run_wbf(result)
#             print(boxes)
#             print(scores)
#             print(labels)
            if not boxes[0]:
                boxes = [[[0, 0, 512, 512]]]
                scores = [[1]]
                labels = [[0]]
#                 print(boxes)
#                 print(scores)
#                 print(labels)
            result = []
            predicted_boxes.extend(boxes)
            predicted_labels.extend(labels)
            predicted_confidences.extend(scores)
            truth_image_ids.append(image_ids[i])
            
stats = get_coco_stats(
        prediction_image_ids=truth_image_ids,
        predicted_class_confidences=predicted_confidences,
        predicted_bboxes=predicted_boxes,
        predicted_class_labels=predicted_labels,
        target_image_ids=truth_image_ids,
        target_bboxes=truth_boxes,
        target_class_labels=truth_labels,
    )['All']

In [58]:
a, b = [], []
for image_id, labels, boxes, confidences in zip(truth_image_ids, predicted_labels, predicted_boxes, predicted_confidences):
    if len(labels) > 0:
        if len(labels) > 1:
            for lab, bx, cf in zip(labels, boxes, confidences):
                a.append(image_id)
                a.append(lab)
                a.append(bx)
                a.append(cf)
                b.append(a)
                a = []
        else:
            a.append(image_id)
            a.append(labels[0])
            a.append(boxes[0])
            a.append(confidences[0])
            b.append(a)
            a = []
    else:
        a.append(image_id)
        a.append(0)
        a.append([0, 0, 0, 0])
        a.append(0)
        b.append(a)
        a=[]
        
predicted_df = pd.DataFrame(b, columns=['image_name', 'label', 'box', 'score'])
predicted_df[['xmin', 'ymin', 'xmax', 'ymax']] = pd.DataFrame(predicted_df['box'].to_list(), index=predicted_df.index)
predicted_df.drop(columns='box', inplace=True)
predicted_df.head()

In [59]:
a, b = [], []
for image_id, labels, boxes in zip(truth_image_ids, truth_labels, truth_boxes):
    if len(labels) > 0:
        if len(labels) > 1:
            for lab, bx in zip(labels, boxes):
                a.append(image_id)
                a.append(lab)
                a.append(bx)
                b.append(a)
                a = []
        else:
            a.append(image_id)
            a.append(labels[0])
            a.append(boxes[0])
            b.append(a)
            a = []
#     else:
#         a.append(image_id)
#         a.append(0)
#         a.append([0, 0, 0, 0])
#         b.append(a)
#         a=[]
        
truth_df = pd.DataFrame(b, columns=['image_name', 'label', 'box'])
truth_df[['xmin', 'ymin', 'xmax', 'ymax']] = pd.DataFrame(truth_df['box'].to_list(), index=truth_df.index)
truth_df.drop(columns='box', inplace=True)
truth_df.head()

In [52]:
from objdetecteval.metrics.iou import iou
import pandas as pd
from typing import List

__all__ = [
    "get_inference_metrics",
    "summarise_inference_metrics",
    "match_preds_to_targets",
    "get_inference_metrics_from_df"
]


def get_inference_metrics_from_df(predictions_df, labels_df):

    matched_bounding_boxes = match_preds_to_targets(
        predictions_df, labels_df
    )

    return get_inference_metrics(**matched_bounding_boxes)


def get_unique_image_names(predictions_df, labels_df):
    # get unique image names from both preds and labels
    # need from both to capture images where there were no predictions
    # and images where there were predictions but no labels
    unique_preds_images = predictions_df['image_name'].unique().tolist()
    unique_label_images = labels_df['image_name'].unique().tolist()
    unique_images = sorted(list(set([*unique_preds_images, *unique_label_images])))
    return unique_images


def match_preds_to_targets(predictions_df, labels_df):

    # check for required df columns
    pred_required_columns = ['image_name', 'xmin', 'xmax', 'ymin', 'ymax', 'label', 'score']
    assert all(col in predictions_df.columns for col in pred_required_columns), \
        f"missing or different column names - should be: {pred_required_columns}"
    label_required_columns = ['image_name', 'xmin', 'xmax', 'ymin', 'ymax', 'label']
    assert all(col in labels_df.columns for col in label_required_columns), \
        f"missing or diferent column names - should be {label_required_columns}"

    image_names = []
    predicted_class_labels = []
    predicted_bboxes = []
    predicted_class_confidences = []
    target_class_labels = []
    target_bboxes = []
    image_preds = {}

    unique_images = get_unique_image_names(predictions_df, labels_df)

    # index the dataframes by the image_name
    preds_df_indexed = predictions_df.set_index('image_name')
    labels_df_indexed = labels_df.set_index('image_name')

    # loop through individual images
    for image_name in unique_images:

        # get the predictions and labels for each image
        preds = preds_df_indexed.loc[image_name]
        labels = labels_df_indexed.loc[image_name]

        # create lists for all the bounding boxes, labels and scores
        # for the image, pascal boxes
        # [[xmin, ymin, xmax, ymax], []]
        # [label, label]
        # [score, score]
        pred_image_bboxes = preds[['xmin', 'ymin', 'xmax', 'ymax']].values.tolist()
        pred_image_class_labels = preds[['label']].values.tolist()
        pred_image_class_confs = preds[['score']].values.tolist()

        # add the predictions lists for the image
        image_names.append(image_name)
        predicted_class_labels.append(pred_image_class_labels)
        predicted_class_confidences.append(pred_image_class_confs)
        if isinstance(pred_image_bboxes[0], list):
          predicted_bboxes.append(pred_image_bboxes)
        else:
          predicted_bboxes.append([pred_image_bboxes])
        # create lists of the label bboxes and classes
        labels_image_bboxes = labels[['xmin', 'ymin', 'xmax', 'ymax']].values.tolist()
        labels_image_class_labels = labels[['label']].values.tolist()

        # add the label lists for the image
        target_class_labels.append(labels_image_class_labels)
        if isinstance(labels_image_bboxes[0], list):
          target_bboxes.append(labels_image_bboxes)
        else:
          target_bboxes.append([labels_image_bboxes])
        

    return {
        "image_ids": image_names,
        "predicted_class_labels": predicted_class_labels,
        "predicted_bboxes": predicted_bboxes,
        "predicted_class_confidences": predicted_class_confidences,
        "target_class_labels": target_class_labels,
        "target_bboxes": target_bboxes
    }



def calc_iou(pred_bbox, true_bboxes):
    iou_val = 0.0
    for true_bbox in true_bboxes:
        # assumes pascal
        box_iou_val = iou(pred_bbox, true_bbox)
        if box_iou_val > iou_val:
            iou_val = box_iou_val
    return iou_val


def calculate_detections(
    all_image_ids,
    all_pred_classes,
    all_pred_bboxes,
    all_pred_confs,
    all_true_classes,
    all_true_bboxes,
    do_iou_calc=True,
):
    assert len(all_image_ids) == len(all_pred_bboxes) == len(all_true_bboxes)

    # ["image_id", "class", "TP", "TN", "FP", "FN", "Confidence", "IoU"]
    detections = []

    for image_id, pred_classes, pred_boxes, pred_confs, true_classes, true_boxes in zip(
        all_image_ids,
        all_pred_classes,
        all_pred_bboxes,
        all_pred_confs,
        all_true_classes,
        all_true_bboxes,
    ):
        # loop through the predicted boxes for the image
        for pred_class, pred_box, pred_conf in zip(
            pred_classes, pred_boxes, pred_confs
        ):
            if type(pred_class) == list:
              pred_class = pred_class[0]
              pred_conf = pred_conf[0]
            if pred_class in true_classes:
                if do_iou_calc:
                    box_iou = calc_iou(pred_box, true_boxes)
                    detections.append(
                        [image_id, pred_class, 1, 0, 0, 0, pred_conf, box_iou]
                    )
                else:
                    # true positive
                    detections.append([image_id, pred_class, 1, 0, 0, 0, pred_conf, -1])

                continue

            if pred_class not in true_classes:
                # false positive
                detections.append([image_id, pred_class, 0, 0, 1, 0, pred_conf, -1])
                continue

        # false negatives
        for true_class in true_classes:
            if true_class not in pred_classes:
                detections.append([image_id, true_class, 0, 0, 0, 1, 0, -1])

    return detections


def summarise_inference_metrics(inference_df):

    class_stats = inference_df.groupby("class")[["TP", "FP", "FN"]].sum()

    # total number for each class
    class_stats["Total"] = class_stats[["TP", "FP", "FN"]].sum(axis=1)

    class_stats["Precision"] = class_stats["TP"] / (
        class_stats["TP"] + class_stats["FP"]
    )
    class_stats["Recall"] = class_stats["TP"] / (class_stats["TP"] + class_stats["FN"])

    # remove the index creatd by the groupby so the class is a column
    class_stats = class_stats.reset_index()

    return class_stats


def get_inference_metrics(
    image_ids: List[int],
    predicted_class_labels: List[List[int]],
    predicted_bboxes: List[List[List[float]]],
    predicted_class_confidences: List[List[float]],
    target_class_labels: List[List[int]],
    target_bboxes: List[List[List[float]]],
):
    """
    Create metrics that do not include IoU. IoU is calculated but is not used to calculate precision and recall.
    Converts the outputs from the models into inference dataframes containing evaluation metrics such as
    precision and recall, and TP, FP, FN, confidence. Useful for more detailed analysis of results and plotting.
    :param image_ids: A list of image ids for each image in the order of the prediction and target lists
    :param predicted_class_labels: A list containing a list of class labels predicted per image
    :param predicted_class_confidences: A list containing a list of prediction confidence values per image
    :param predicted_bboxes: A list containing a list of bounding boxes, in Pascal VOC format, predicted per image
    :param target_class_labels: A list containing a list of ground truth class labels per image
    :param target_bboxes: A list containing a list of ground truth bounding boxes, in Pascal VOC format
    :param conv_bbox_func: A function to convert the format of incoming bboxes to pascal format, default is None
    :returns: a DataFrame of the results, and a dataframe containing precision and recall.
    """

    detections = calculate_detections(
        image_ids,
        predicted_class_labels,
        predicted_bboxes,
        predicted_class_confidences,
        target_class_labels,
        target_bboxes,
    )

    inference_df = pd.DataFrame(
        detections,
        columns=["image_id", "class", "TP", "TN", "FP", "FN", "Confidence", "IoU"],
    )

    return inference_df

In [60]:
from objdetecteval.metrics import (
    image_metrics as im,
    coco_metrics as cm
)
infer_df = get_inference_metrics_from_df(predicted_df, truth_df)
infer_df.head()

In [61]:
class_summary_df = im.summarise_inference_metrics(infer_df)
class_summary_df

In [ ]:
import matplotlib.pyplot as plt


#predictions = make_predictions(images)

#i = 0
sample = test_dataset[0][0].permute(1,2,0).cpu().numpy()

boxes, scores, labels = run_wbf(result, image_index=0)
boxes = boxes.astype(np.int32).clip(min=0, max=511)
fig, ax = plt.subplots(1, 1, figsize=(16, 8))

for box in boxes:
    cv2.rectangle(sample, (box[0], box[1]), (box[2], box[3]), (1, 0, 0), 1)
    
ax.set_axis_off()
ax.imshow(sample);